In [1]:
import os
import pandas as pd
import xlrd
import time
from bisect import bisect_right
from bisect import bisect_left
from deep_translator import GoogleTranslator
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np
import pandas as pd

Loading the already cleaned OECD dataset

In [2]:
df = pd.read_excel("OECD_All_v3.xlsx", sheet_name = "All")

#### 2)Encoding "Description" and "Project Title":

2.1) Substituting all blank values (NaN) in "Description" and "Project Title" with "hello world"

In [3]:
df["Description"] = df["Description"].fillna("hello_world")
df["Project Title"] = df["Project Title"].fillna("hello_world")

2.2) Translating all non-english entries in "Description" and "Project title" to English.

In [4]:
ptitle = df["Project Title"].astype(str).tolist()
desc = df["Description"].astype(str).tolist()

In [5]:
title_desc = list(map(lambda x,y: x + " /// " + y, ptitle, desc))

In [15]:
with open("title_desc_152768.txt", "r+", encoding="utf-8") as td_file:    
    td_content = td_file.read()

In [16]:
title_desc_en = td_content.split("_$%$&_")

In [17]:
len(title_desc_en)

152768

In [18]:
for j in range(152768, len(title_desc)):
    if j%500 == 0:
        print(j)
    td_en = GoogleTranslator('auto', 'english').translate(title_desc[j])
    title_desc_en.append(td_en)

153000
153500
154000
154500
155000
155500
156000
156500
157000
157500
158000
158500
159000
159500
160000
160500
161000
161500
162000
162500
163000
163500
164000
164500
165000
165500
166000
166500
167000


In [19]:
len(title_desc_en)

167363

In [21]:
len(title_desc)

167363

In [22]:
text_to_file = "_$%$&_".join(title_desc_en)

In [23]:
with open("title_desc_167363.txt", "a", encoding="utf-8") as f:
    f.write(text_to_file)
f.close()

2.3) Concatenating descriptions and titles lists

In [ ]:
determiners = "the, a, an, this, that, these, those, my, your, his, her, its, our, their, few, little, much, many, lot, most, some, any, enough, all, both, half, either, neither, each, every, other, another, such, what, rather, quite"
conjunctions = "and, or, but, yet, because, since, as, when, before, after, once, until, whenever, while, where, wherever, if, unless, case, although, though, whereas"
prepositions = "in, at, on, for, from, to, until, by, towards, across, through, between, among, of, with, by, about"

In [ ]:
det_list = determiners.split(", ")
conj_list = conjunctions.split(", ")
prep_list = prepositions.split(", ")
num_list = [str(i) for i in range(0,3000)]

In [ ]:
det_list_up = [i.capitalize() for i in det_list]
conj_list_up = [i.capitalize() for i in conj_list]
prep_list_up = [i.capitalize() for i in prep_list]

In [ ]:
all_words = det_list + conj_list + prep_list + num_list + ["hello_world"]

In [ ]:
vectorizer = CountVectorizer(input = "content", lowercase = "True", stop_words = all_words, analyzer = "word")
text_x = vectorizer.fit_transform(title_desc_en)


In [ ]:
text_mtx = list(text_x.toarray())


In [ ]:
text_df = pd.DataFrame(text_mtx)


In [ ]:
text_shp = text_df.shape()


In [ ]:
text_col = text_shp[1]
text_colnames = ["Text_" + str(i) for i in range(0, text_col)]


In [ ]:
text_df.columns = text_colnames


In [151]:
encoded_df2 = encoded_df.drop(["Description", "Project Title"], axis = 1)

In [ ]:
encoded_df3 = pd.concat([encoded_df2, text_df], axis = 1)

#### 3) Encoding the rest of the fields and the Rio labels:

3.1) Substitute blank values (NaN) with 0s and -1s (only for the funds related fields).

In [152]:
funds_fields = ['Adaptation-related development finance - Commitment - 2021 USD thousand',
       'Mitigation-related development finance - Commitment - 2021 USD thousand',
       'Overlap - Commitment - 2021 USD thousand',
       'Climate-related development finance - Commitment - 2021 USD thousand']

In [153]:
no_funds_fields = [field for field in df.columns if field not in funds_fields]

In [154]:
df[no_funds_fields] = df[no_funds_fields].fillna(0)

In [155]:
df[funds_fields] = df[funds_fields].fillna(-1)

3.2) In the non-numeric fields, assign a number to each unique element.

In [156]:
non_numeric_fields = ['Provider Type', 'Provider', 'Provider (detailed)', 'Extending Agency', 'CRS Identification N°',
 'Donor project N°', 'Recipient Region', 'Recipient Income Group (OECD Classification)',  'Concessionality', 'Channel of Delivery', 'Sector (detailed)',
 'Sub-sector', 'Development Cooperation Modality', 'Financial Instrument', 'Gender']

In [157]:
encoded_fields_dic = {}
for field in non_numeric_fields:
    f_list = df[field].tolist()
    items = list(filter(lambda x:x!=0,f_list))
    dic = {num:item for item, num in enumerate(set(items),1)}
    dic[0] = 0
    encoded_name = field + "_encoded"
    encoded_fields_dic[encoded_name] = [dic[item] for item in f_list]

In [158]:
nnumeric_encoded_df = pd.DataFrame.from_dict(encoded_fields_dic, orient = "columns")

3.3) Encode the Rio marker labels.

In [159]:
label_fields = ['Climate objective (applies to Rio-marked data only) or climate component',
 'Adaptation objective (applies to Rio-marked data only)', 'Mitigation objective (applies to Rio-marked data only)']

In [160]:
encoded_labels_dic ={}
labels_dic = {"Principal" : 4, "Significant" : 3,  
                      "Climate components" : 2, "Imputed multilateral contributions" : 1, "Not targeted/Not screened" : 0}
for field in label_fields:
    f_list = df[field].tolist()
    items = list(filter(lambda x:x!=0,f_list))
    encoded_name = field + "_encoded"
    encoded_labels_dic[encoded_name] = [labels_dic[item] for item in f_list]

In [161]:
labels_encoded_df = pd.DataFrame.from_dict(encoded_labels_dic, orient = "columns")

3.4) Join the enconded fields and labels data frames.

In [164]:
df1 = df.drop(columns = non_numeric_fields + label_fields)
encoded_df = pd.concat([df1,nnumeric_encoded_df, labels_encoded_df], axis = 1)